In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('/Users/sanchitwani/Documents/Insurance Fraud/Worksheet in Case Study question 2.xlsx')

In [3]:
data.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N


In [4]:
data.shape

(1000, 39)

### If someone claims more than they paid in premiums, it may indicate suspicious behavior. This is normalized, unlike total claim amount which favors high-premium policies. Low risk of leakage, high interpretability.

In [5]:
data['claim_severity_ratio'] = data['total_claim_amount'] / data['policy_annual_premium']

### Helps identify unusual claim structures (e.g., 100% injury, no property damage).This can be predictive since fraudsters may inflate one part disproportionately.

In [6]:
data['injury_ratio'] = data['injury_claim'] / data['total_claim_amount']
data['property_ratio'] = data['property_claim'] / data['total_claim_amount']
data['vehicle_ratio'] = data['vehicle_claim'] / data['total_claim_amount']

In [7]:
# List of the ratio columns
ratio_cols = ['injury_ratio', 'property_ratio', 'vehicle_ratio']

# Calculate the variance of each column
variances = data[ratio_cols].var()

# Find the column name with the minimum variance
col_to_drop = variances.idxmin()

# Drop the column from the DataFrame
data.drop(columns=col_to_drop, inplace=True)

print(f"Dropped column with the lowest variance: '{col_to_drop}'")

Dropped column with the lowest variance: 'property_ratio'


In [8]:
# A quick check you can run
print(data[['injury_ratio', 'vehicle_ratio']].isnull().sum())

injury_ratio     0
vehicle_ratio    0
dtype: int64


In [9]:
from dateutil.relativedelta import relativedelta

# Function to calculate the number of full months between bind and incident
def from_bind_to_incident_months(row):
    diff = relativedelta(row['incident_date'], row['policy_bind_date'])
    return diff.years * 12 + diff.months

# Ensure date columns are datetime
data['policy_bind_date'] = pd.to_datetime(data['policy_bind_date'])
data['incident_date'] = pd.to_datetime(data['incident_date'])

# Apply the function to compute safe tenure
data['from_bind_to_incident_months'] = data.apply(from_bind_to_incident_months, axis=1)

In [10]:
data.columns

Index(['months_as_customer', 'age', 'policy_number', 'policy_bind_date',
       'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital-gains', 'capital-loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported', 'claim_severity_ratio',
       'injury_ratio', 'vehicle_ratio', 'from_bind_to_incident_months'],
      dtype='object')

In [11]:
data.shape

(1000, 43)

In [12]:
cols_to_drop = [
    'policy_state',
    'incident_city',
    'incident_location',
    'auto_model',
    'insured_education_level',
    'policy_number',
    'insured_zip',
    'auto_year',
    'injury_claim',
    'property_claim',
    'vehicle_claim',
    'policy_bind_date',
    'incident_date'
]

In [13]:
data.drop(columns=cols_to_drop, inplace=True)

In [14]:
data.shape

(1000, 30)

In [15]:
# Convert to categorical type
data['policy_deductable'] = data['policy_deductable'].astype('category')

# Optional: verify
print(data['policy_deductable'].value_counts())
print(data['policy_deductable'].dtype)

policy_deductable
1000    351
500     342
2000    307
Name: count, dtype: int64
category


In [16]:
# Check for negative values
print("Negative values before fix:", (data['umbrella_limit'] < 0).sum())

# Fix the known incorrect value
data['umbrella_limit'] = data['umbrella_limit'].abs()

# Optional: confirm it's fixed
print("Negative values after fix:", (data['umbrella_limit'] < 0).sum())

Negative values before fix: 1
Negative values after fix: 0


In [17]:
data.shape

(1000, 30)

In [18]:
data['collision_type'].unique()

array(['Side Collision', '?', 'Rear Collision', 'Front Collision'],
      dtype=object)

In [19]:
data['property_damage'].unique()

array(['YES', '?', 'NO'], dtype=object)

In [20]:
# Replace '?' with 'Unknown' in both columns
data['collision_type'] = data['collision_type'].replace('?', 'Unknown')
data['property_damage'] = data['property_damage'].replace('?', 'Unknown')
data['police_report_available'] = data['police_report_available'].replace('?', 'Unknown')

# Optional: check counts
print(data['collision_type'].value_counts())
print(data['property_damage'].value_counts())
print(data['police_report_available'].value_counts())

collision_type
Rear Collision     292
Side Collision     276
Front Collision    254
Unknown            178
Name: count, dtype: int64
property_damage
Unknown    360
NO         338
YES        302
Name: count, dtype: int64
police_report_available
Unknown    343
NO         343
YES        314
Name: count, dtype: int64


### Try adding authorities_contacted column to the data later

In [21]:
data.shape

(1000, 30)

In [22]:
# Select only categorical columns
cat_cols = data.select_dtypes(include=['object', 'category']).columns

# Print number of unique values in each categorical column
for col in cat_cols:
    print(f"{col}: {data[col].nunique()} unique values")

policy_csl: 3 unique values
policy_deductable: 3 unique values
insured_sex: 2 unique values
insured_occupation: 14 unique values
insured_hobbies: 20 unique values
insured_relationship: 6 unique values
incident_type: 4 unique values
collision_type: 4 unique values
incident_severity: 4 unique values
authorities_contacted: 4 unique values
incident_state: 7 unique values
property_damage: 3 unique values
police_report_available: 3 unique values
auto_make: 14 unique values
fraud_reported: 2 unique values


In [23]:
# Define the list of hobbies you want to flag
high_fraud_hobbies = ['chess', 'cross-fit']

# Create a new column 'is_high_risk_hobby'
data['is_high_risk_hobby'] = data['insured_hobbies'].isin(high_fraud_hobbies).astype(int)


# Print the top 5 rows of the original and new column to see the result
print("Original hobbies vs. new binary feature:")
print(data[['insured_hobbies', 'is_high_risk_hobby']].head())

# Check the value counts of the new column
print("\nValue counts for 'is_high_risk_hobby':")
print(data['is_high_risk_hobby'].value_counts())

Original hobbies vs. new binary feature:
  insured_hobbies  is_high_risk_hobby
0        sleeping                   0
1         reading                   0
2     board-games                   0
3     board-games                   0
4     board-games                   0

Value counts for 'is_high_risk_hobby':
is_high_risk_hobby
0    919
1     81
Name: count, dtype: int64


In [24]:
data.drop(columns='insured_hobbies', inplace=True)

In [25]:
data['fraud_reported'] = data['fraud_reported'].map({'Y': 1, 'N': 0})

In [26]:
data['fraud_reported'].head()

0    1
1    1
2    0
3    1
4    0
Name: fraud_reported, dtype: int64

In [27]:
data['incident_severity'].unique()

array(['Major Damage', 'Minor Damage', 'Total Loss', 'Trivial Damage'],
      dtype=object)

In [28]:
# Create a dictionary to define the mapping
severity_mapping = {
    'Trivial Damage': 0,
    'Minor Damage': 1,
    'Major Damage': 2,
    'Total Loss': 3
}

# Apply the mapping to the 'incident_severity' column
data['incident_severity_encoded'] = data['incident_severity'].map(severity_mapping)

# --- Verification ---
# Display the original and newly encoded columns
print("Original vs. Encoded Column:")
print(data[['incident_severity', 'incident_severity_encoded']].head(10))

# Check the value counts of the new column
print("\nValue counts for the new encoded column:")
print(data['incident_severity_encoded'].value_counts())

Original vs. Encoded Column:
  incident_severity  incident_severity_encoded
0      Major Damage                          2
1      Minor Damage                          1
2      Minor Damage                          1
3      Major Damage                          2
4      Minor Damage                          1
5      Major Damage                          2
6      Minor Damage                          1
7        Total Loss                          3
8        Total Loss                          3
9        Total Loss                          3

Value counts for the new encoded column:
incident_severity_encoded
1    354
3    280
2    276
0     90
Name: count, dtype: int64


In [29]:
data.drop(columns='incident_severity', inplace=True)

In [30]:
# Select only categorical columns
cat_cols = data.select_dtypes(include=['object', 'category']).columns

# Print number of unique values in each categorical column
for col in cat_cols:
    print(f"{col}: {data[col].nunique()} unique values")

policy_csl: 3 unique values
policy_deductable: 3 unique values
insured_sex: 2 unique values
insured_occupation: 14 unique values
insured_relationship: 6 unique values
incident_type: 4 unique values
collision_type: 4 unique values
authorities_contacted: 4 unique values
incident_state: 7 unique values
property_damage: 3 unique values
police_report_available: 3 unique values
auto_make: 14 unique values


In [31]:
# This assumes your DataFrame is named 'data'
data['claim_x_tenure'] = data['total_claim_amount'] * data['from_bind_to_incident_months']

In [32]:
data.columns

Index(['months_as_customer', 'age', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_sex',
       'insured_occupation', 'insured_relationship', 'capital-gains',
       'capital-loss', 'incident_type', 'collision_type',
       'authorities_contacted', 'incident_state', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'auto_make', 'fraud_reported', 'claim_severity_ratio', 'injury_ratio',
       'vehicle_ratio', 'from_bind_to_incident_months', 'is_high_risk_hobby',
       'incident_severity_encoded', 'claim_x_tenure'],
      dtype='object')

In [34]:
data.to_csv('/Users/sanchitwani/Documents/Insurance Fraud/3 Try/cleaned_3_insurance.csv', index=False)